# Preprocessing

This notebook is used to process the raw transcripts into a format that can then be used for model training and testing.


In [1]:
#!/usr/bin/env/python
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import string
import unicodedata
import matplotlib
import matplotlib.pyplot as plt
import re

In [2]:
#Load Subset 1 and Subset 2
path1 = '../data/Subset1_WithDialogActs/'
subdirs = glob.glob('%s*'%(path1))

path2 = '../data/Subset2_WithDialogActs/'
subdirs.extend(glob.glob('%s*'%(path2)))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [3]:
'''
To avoid overfitting the models we carefully divide the training and testing set. 
Testing set include some teachers who particiapted in Year 1. The transcripts from these teachers are not included
in the training set. The teachers in this list were picked at random.
'''

train_subdirs =[]
test_subdirs = []
for item in subdirs:
    if item.find("Travers.Spring") != -1 or item.find("Barry.Spring") != -1 \
    or item.find("Carroll") != -1 or item.find("Saunders.Spring") != -1 \
    or item.find("Keene") != -1 or item.find("Carter") != -1 or item.find("Basker") != -1 :
        test_subdirs.append(item)
    else:
        train_subdirs.append(item)
print(len(train_subdirs), len(test_subdirs))

503 63


In [4]:
data = None
temp = []
flag = 0
for dir in train_subdirs: #replace with test_subdirs to generate test data
    if dir.endswith('.xlsx') and not dir.startswith('~'):
        try:
            temp_data = None
            load_data = pd.read_excel(dir)
            load_data['Sentence'] = load_data['Sentence'].str.replace("[\(\[].*?[\)\]]", "", regex=True)
            load_data = load_data[load_data['Sentence'] != ""]
            mask = load_data['Sentence'].str.contains('(.*[\(].*|.*[\)].*|.*[\[].*|.*[\]].*)|(^[.?!]+$)|(^.*[0-9]{1,2}:[0-9]{1,2}:[0-9]{1,2}.*$)',regex=True,na=False)
            temp_data = load_data[:][~mask]
            temp_data = temp_data.loc[:, ~temp_data.columns.str.contains('^Unnamed')]
            temp_data['Transcript'] = dir.split('/')[len(dir.split('/'))-1]
            for item in temp_data.columns:
                if item not in ['Teacher Tag', 'TimeStamp' ,'Turn','Student Tag','Sentence','Speaker','Transcript','DAMSLTag']:
                    temp_data = temp_data.drop(columns=[item])
            if flag == 0:
                data = temp_data
                flag = 1
            else:
                if len(data.columns) > 8:
                    print(data.columns,dir)
                data = data.append(temp_data, ignore_index=True,sort=False)
        except Exception as e:
            print(e,dir)
            pass 

/home/abhijit/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [5]:
data = data[(data['DAMSLTag'].notna()) & (data['DAMSLTag'] != '%')]
print(data.head(), set(data['DAMSLTag']))

  TimeStamp Turn Speaker                                           Sentence  \
0       NaN  NaN       T               Do you want to say it to the camera?   
1       NaN  NaN    Jeff                        No I don’t want to say it.    
2       NaN  NaN    Jeff  This is going to be the number in the square m...   
3       NaN  NaN       T  Yeah, what have you written here, that’s very ...   
4       NaN  NaN    Jeff               Zero times zero plus one equals one.   

                     Teacher Tag                                 Student Tag  \
0  2 - keeping everyone together                                         NaN   
1                            NaN                                    1 - none   
2                            NaN                          4 - making a claim   
3                       1 - none                                         NaN   
4                            NaN  5 - providing evidence/providing reasoning   

  DAMSLTag                                  

In [6]:
dataset = []
bad_sentences = []
bad_speakers = []
num_sentences = []
bad_entries = []
translator = str.maketrans('', '', string.punctuation)
for entry in data.iterrows():
    try:
        if type(entry[1]['Speaker']) == int or entry[1]['Speaker'] == float:
            bad_speakers.append(entry[1]['Speaker'])
            continue
        if repr(type(entry[1]['Sentence'])) == "<type 'datetime.time'>" or repr(type(entry[1]['Sentence'])) == "<type 'datetime.datetime'>" or repr(entry[1]['Speaker']) == 'nan':
            bad_sentences.append(entry[1]['Sentence'])
            continue
    
        if repr(type(entry[1]['Sentence'])) == "<type 'int'>" or repr(type(entry[1]['Sentence'])) == "<type 'float'>":
            num_sentences.append(str(entry[1]['Sentence']))
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
        if repr(type(entry[1]['Sentence'])) == "<type 'str'>":
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
            
        temp = unicodedata.normalize('NFKD', str(entry[1]['Sentence']).translate(translator))
        
        # Turn, Speaker, Sentence, Teacher Tag, Student Tag, Dialog Act tag
        try:
            tag = str(entry[1]['Teacher Tag']).split('-')[0].strip()
        except Exception as e:
            print(e,'####')
        try:
            stag = str(entry[1]['Student Tag']).split('-')[0].strip()
        except Exception as e:
            print(e,'####')
        
        sent = ' '.join([re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in temp.split(" ")])
        
        if str(stag) != 'nan':
            #print([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['DAMSLTag'],entry[1]['Transcript']])
        elif str(tag) != 'nan':
            #print([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['DAMSLTag'],entry[1]['Transcript']])
       
    except Exception as e:
        print(e)
        bad_entries.append([entry[1]['Sentence'], entry[1]['Speaker']])
        continue

In [7]:
df = pd.DataFrame({'Turn':[i[0] for i in dataset], 'Speaker':[i[1] for i in dataset], \
                   'Sentence':[i[2] for i in dataset], 'Tag':[i[3] for i in dataset],\
                   'StudentTag':[i[4] for i in dataset], 'DAMSLTag':[i[5] for i in dataset],
                   'Transcript':[i[6] for i in dataset]})
df = df.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df

,Turn,Speaker,Sentence,Tag,StudentTag,DAMSLTag,Transcript
0,NaN,T,Do you want to say it to the camera,2,nan,sd,Video Mosaic Grade 6 2 variables 14.xlsx_with_...
1,NaN,S,No I don t want to say it,nan,1,sd,Video Mosaic Grade 6 2 variables 14.xlsx_with_...
2,NaN,S,This is going to be the number in the square m...,nan,4,sd,Video Mosaic Grade 6 2 variables 14.xlsx_with_...
3,NaN,T,Yeah what have you written here that s very in...,1,nan,sd,Video Mosaic Grade 6 2 variables 14.xlsx_with_...
4,NaN,S,Zero times zero plus one equals one,nan,5,sd,Video Mosaic Grade 6 2 variables 14.xlsx_with_...
...,...,...,...,...,...,...,...
191768,166,T,Yes,1,nan,b,TalkBack.Year1.Wolfram.Spring.011420-1.xlsx_wi...
191769,167,T,zero or a yintercept,1,nan,sd,TalkBack.Year1.Wolfram.Spring.011420-1.xlsx_wi...
191770,167,T,You said its undefined where it crosses the yaxis,1,nan,sd,TalkBack.Year1.Wolfram.Spring.011420-1.xlsx_wi...
191771,167,T,Then that must be your vertical asymptote,1,nan,sd,TalkBack.Year1.Wolfram.Spring.011420-1.xlsx_wi...


In [8]:
writer = pd.ExcelWriter('../data/test_data_63_dailog_acts.xlsx') #replace with test_data_63 for test data
df.to_excel(writer)
writer.save()

In [ ]:
'''
Pre-processing 

- All lower cased
- All punctuation removed
- Converted all teachers to T tags
- Converted all students to S tags
- T labels range from  [0-6] - 0 stands for None
- S are labelled None
- Compiled all the data into single excel sheet
- At present there are 91891 sentences in total

Cleaning

- [End of Class], [Class Ends] - Sentence in brackets
- If Sentence contains a timestamp or date time stamp
- If sentence is just . or ! or ? or empty
- Speaker is a number (All of them are nans)
- Speaker is nan


'''

In [10]:
#Prepare Tuples from data - Student
data = pd.read_excel('../data/test_data_63_dailog_acts.xlsx') #replace with test_data_63 to generate test data
empty_previous = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        continue
    if row[1]['Speaker'] == 'S' and flag == 0:
        temp =  row[1]['Sentence']
        flag=1
        sentence_tuple.append([empty_previous,temp,row[1]['DAMSLTag'],row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['DAMSLTag'], row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        temp =  row[1]['Sentence']
        continue
        
df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple], \
                   'text_b':[i[1] for i in sentence_tuple], \
                   'damsl_labels':[i[2] for i in sentence_tuple],
                   'labels':[i[3] for i in sentence_tuple]})

df.to_csv('../data/test_student_dact.tsv',sep='\t',index=False)   #replace with test_student to generate test data  

In [9]:
#Prepare Tuples from data - teacher
data = pd.read_excel('../data/test_data_63_dailog_acts.xlsx') #replace with test_data_63 to generate test data
empty_student = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'S' and flag == 2:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 2
        continue
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['DAMSLTag'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['DAMSLTag'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 2:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['DAMSLTag'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        flag = 1
        continue

df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple],\
                   'text_b':[i[1] for i in sentence_tuple],\
                   'damsl_labels':[i[2] for i in sentence_tuple],\
                   'labels':[i[3] for i in sentence_tuple]})

df.to_csv('../data/test_teacher_dact.tsv',sep='\t',index=False)  #replace with test_teacher to generate test data 